<a href="https://colab.research.google.com/github/Manasa-Raghavendra/Internship_tasks/blob/main/finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Sun Aug 31 11:49:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install transformers datasets accelerate torch


In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Set pad token to eos_token
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

# Move model to GPU
model = model.to("cuda")

print("✅ GPT-2 and tokenizer loaded successfully!")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

✅ GPT-2 and tokenizer loaded successfully!


In [ ]:
from google.colab import files
files.upload()  # Upload archive.zip

!unzip archive.zip


Saving archive.zip to archive.zip
Archive:  archive.zip
  inflating: writingPrompts/README   
  inflating: writingPrompts/test.wp_source  
  inflating: writingPrompts/test.wp_target  
  inflating: writingPrompts/train.wp_source  
  inflating: writingPrompts/train.wp_target  
  inflating: writingPrompts/valid.wp_source  
  inflating: writingPrompts/valid.wp_target  


In [ ]:
N = 5000  # number of samples you want to keep

# Save smaller source file
with open("/content/writingPrompts/train.wp_source") as src, open("/content/train_small_source.txt", "w") as out_src:
    for i, line in enumerate(src):
        if i >= N: break
        out_src.write(line)

# Save smaller target file
with open("/content/writingPrompts/train.wp_target") as tgt, open("/content/train_small_target.txt", "w") as out_tgt:
    for i, line in enumerate(tgt):
        if i >= N: break
        out_tgt.write(line)


In [ ]:
!wc -l /content/train_small_source.txt
!wc -l /content/train_small_target.txt


5000 /content/train_small_source.txt
5000 /content/train_small_target.txt


In [ ]:
!zip small_dataset.zip train_small_source.txt train_small_target.txt


updating: train_small_source.txt (deflated 62%)
updating: train_small_target.txt (deflated 64%)


In [ ]:
combined_path = "/content/train_small_combined.txt"

with open("/content/train_small_source.txt") as src, open("/content/train_small_target.txt") as tgt, open(combined_path, "w") as out:
    for s, t in zip(src, tgt):
        out.write("[WP] " + s.strip() + " " + t.strip() + "\n")

print("✅ Combined dataset saved at:", combined_path)
!head -3 {combined_path}  # Show first 3 samples


✅ Combined dataset saved at: /content/train_small_combined.txt
[WP] [ WP ] You 've finally managed to discover the secret to immortality . Suddenly , Death appears before you , hands you a business card , and says , `` When you realize living forever sucks , call this number , I 've got a job offer for you . '' So many times have I walked on ruins , the remainings of places that I loved and got used to.. At first I was scared , each time I could feel my city , my current generation collapse , break into the black hole that thrives within it , I could feel humanity , the way I 'm able to feel my body.. After a few hundred years , the pattern became obvious , no longer the war and damage that would devastate me over and over again in the far past was effecting me so dominantly . <newline> It 's funny , but I felt as if after gaining what I desired so long , what I have lived for my entire life , only then , when I achieved immortality I started truly aging . <newline> <newline> 5 world w

In [ ]:
# ✅ Combine source + target into single train.txt / valid.txt
with open("/content/writingPrompts/train.wp_source", "r", encoding="utf-8") as src, \
     open("/content/writingPrompts/train.wp_target", "r", encoding="utf-8") as tgt, \
     open("train.txt", "w", encoding="utf-8") as out:
    for s, t in zip(src, tgt):
        out.write(s.strip() + " " + t.strip() + "\n")

with open("/content/writingPrompts/valid.wp_source", "r", encoding="utf-8") as src, \
     open("/content/writingPrompts/valid.wp_target", "r", encoding="utf-8") as tgt, \
     open("valid.txt", "w", encoding="utf-8") as out:
    for s, t in zip(src, tgt):
        out.write(s.strip() + " " + t.strip() + "\n")

print("✅ train.txt and valid.txt created")


✅ train.txt and valid.txt created


In [ ]:
from datasets import load_dataset

dataset = load_dataset("text", data_files={"train": combined_path})
print(dataset)


Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 5000
    })
})


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load GPT-2 small
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# GPT2 doesn’t have a pad_token by default, so we fix it
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained("gpt2")
model.config.pad_token_id = tokenizer.eos_token_id


In [ ]:
# Split dataset into 90% train, 10% validation
dataset_split = dataset["train"].train_test_split(test_size=0.1, seed=42)

# Get actual sizes
train_size = min(5000, len(dataset_split["train"]))
valid_size = min(1000, len(dataset_split["test"]))

# Select available samples
train_dataset = dataset_split["train"].select(range(train_size))
valid_dataset = dataset_split["test"].select(range(valid_size))

print("Train samples:", len(train_dataset))
print("Validation samples:", len(valid_dataset))



Train samples: 4500
Validation samples: 500


In [ ]:
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

# Apply tokenizer and remove "text" column
train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=["text"])
valid_dataset = valid_dataset.map(tokenize_function, batched=True, remove_columns=["text"])


Map:   0%|          | 0/4500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
from torch.utils.data import DataLoader
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=data_collator)
valid_dataloader = DataLoader(valid_dataset, batch_size=2, shuffle=False, collate_fn=data_collator)



In [ ]:
import torch
from torch.optim import AdamW

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)

epochs = 1
for epoch in range(epochs):
    print(f"Epoch {epoch+1}")
    model.train()
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}


        outputs = model(**batch)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Loss: {loss.item()}")


Epoch 1


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Loss: 3.2043564319610596


In [ ]:
num_epochs = 3  # try 3–5
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for step, batch in enumerate(train_dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**batch)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()
        if step % 100 == 0:
            print(f"Epoch {epoch+1} | Step {step} | Loss: {loss.item():.4f}")

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1} finished | Avg Train Loss: {avg_train_loss:.4f}")


Epoch 1 | Step 0 | Loss: 2.5476
Epoch 1 | Step 100 | Loss: 3.5141
Epoch 1 | Step 200 | Loss: 2.7616
Epoch 1 | Step 300 | Loss: 2.8129
Epoch 1 | Step 400 | Loss: 2.9276
Epoch 1 | Step 500 | Loss: 2.9986
Epoch 1 | Step 600 | Loss: 2.7662
Epoch 1 | Step 700 | Loss: 2.9175
Epoch 1 | Step 800 | Loss: 2.9731
Epoch 1 | Step 900 | Loss: 3.2761
Epoch 1 | Step 1000 | Loss: 2.6524
Epoch 1 | Step 1100 | Loss: 3.2829
Epoch 1 | Step 1200 | Loss: 2.9844
Epoch 1 | Step 1300 | Loss: 2.9730
Epoch 1 | Step 1400 | Loss: 2.9828
Epoch 1 | Step 1500 | Loss: 3.6623
Epoch 1 | Step 1600 | Loss: 3.0411
Epoch 1 | Step 1700 | Loss: 3.0915
Epoch 1 | Step 1800 | Loss: 2.8785
Epoch 1 | Step 1900 | Loss: 2.5860
Epoch 1 | Step 2000 | Loss: 3.0776
Epoch 1 | Step 2100 | Loss: 2.9597
Epoch 1 | Step 2200 | Loss: 2.6975
Epoch 1 finished | Avg Train Loss: 2.9929
Epoch 2 | Step 0 | Loss: 2.1685
Epoch 2 | Step 100 | Loss: 2.2461
Epoch 2 | Step 200 | Loss: 2.4440
Epoch 2 | Step 300 | Loss: 3.2329
Epoch 2 | Step 400 | Loss: 2.56

In [ ]:
model.eval()
valid_loss = 0

with torch.no_grad():
    for batch in valid_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)   # no need to pass labels again
        valid_loss += outputs.loss.item()

avg_valid_loss = valid_loss / len(valid_dataloader)
print(f"Validation Loss after Epoch {epoch+1}: {avg_valid_loss:.4f}")



Validation Loss after Epoch 3: 3.2947


In [ ]:
model.save_pretrained("./fine_tuned_gpt2")
tokenizer.save_pretrained("./fine_tuned_gpt2")


('./fine_tuned_gpt2/tokenizer_config.json',
 './fine_tuned_gpt2/special_tokens_map.json',
 './fine_tuned_gpt2/vocab.json',
 './fine_tuned_gpt2/merges.txt',
 './fine_tuned_gpt2/added_tokens.json',
 './fine_tuned_gpt2/tokenizer.json')

In [ ]:
# Save model and tokenizer
model.save_pretrained("./fine_tuned_gpt2")
tokenizer.save_pretrained("./fine_tuned_gpt2")

print("✅ Model and tokenizer saved in './fine_tuned_gpt2/'")


✅ Model and tokenizer saved in './fine_tuned_gpt2/'


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load
model = GPT2LMHeadModel.from_pretrained("./fine_tuned_gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("./fine_tuned_gpt2")

model.to(device)
model.eval()


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
prompt = "Once upon a time"
inputs = tokenizer(prompt, return_tensors="pt").to(device)

outputs = model.generate(
    **inputs,
    max_length=100,
    num_return_sequences=1,
    temperature=0.7,
    top_p=0.9,
    do_sample=True
)

print("Generated Text:\n", tokenizer.decode(outputs[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text:
 Once upon a time traveler , you meet a stranger who is very young and very strange . Describe the day you meet him . I was a young man when we first met . It was just another day , another day at the office . I was just sitting in my office , looking out my window and my phone ringing . I looked up at my phone and immediately called my wife . I called again . I 'd been calling for years , and I was finally getting it . I called again ,
